In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], "src"))

## create scene builder

In [2]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None, base_link="base_link")
# s_builder.reset_reference_coord(ref_name="floor")

## init combined robot config

In [3]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7,  ((0,0,0), (0,0,0)),
                INDY_IP)]
              , connection_list=[False])
gscene = s_builder.create_gscene(crob)

connection_list
[False]
Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.


Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]


## init planning scene

In [4]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

### planning pipeline & UI

In [5]:
from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)


## add environment

In [6]:
from pkg.geometry.geometry import *

 * Environment: production


In [7]:
floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (0.6,0.6,0.01), (0.4,0,-0.005), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=False, fixed=True, collision=False)
floor = gscene.create_safe(GEOTYPE.BOX, "floor_viscol", "base_link", (3,3,0.01), (0,0,-0.006), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
goal = gscene.create_safe(GEOTYPE.BOX, "goal", "base_link", (0.1,0.1,0.01), (0.4,0.3,-0.005), 
                          rpy=(0,0,0), color=(0.8,0.8,0.2,1), display=True, fixed=True, collision=False)

   Use a production WSGI server instead.
 * Debug mode: off
Please create a subscriber to the marker


In [8]:
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True, exclude_link=["panda1_link7"])

## add brush

In [9]:
gbox1 = gscene.create_safe(gtype=GEOTYPE.BOX, name="box1", link_name="base_link", 
                                dims=(0.05,0.05,0.05), center=(0.4,-0.3,0.025), rpy=(0,0,0), 
                                color=(0.7,0.3,0.3,1), display=True, collision=True, fixed=False)
gbox2 = gscene.create_safe(gtype=GEOTYPE.BOX, name="box2", link_name="base_link", 
                                dims=(0.05,0.05,0.05), center=(0.4,0.0,0.025), rpy=(0,0,0), 
                                color=(0.3,0.7,0.3,1), display=True, collision=True, fixed=False)
gbox3 = gscene.create_safe(gtype=GEOTYPE.BOX, name="box3", link_name="base_link", 
                                dims=(0.05,0.05,0.05), center=(0.4,0.3,0.025), rpy=(0,0,0), 
                                color=(0.3,0.3,0.7,1), display=True, collision=True, fixed=False)

## Register binders

In [10]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepTool

In [11]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="indy0_tcp", 
                 dims=(0.01,)*3, center=(0,0,0.14), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

In [12]:
pscene.create_binder(bname="grip0", gname="grip0", rname="indy0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane)
pscene.create_binder(bname="goal_bd", gname="goal", _type=PlacePlane, point=(0,0,0.005), rpy=(0,0,0))

## add objects

In [13]:
from pkg.planning.constraint.constraint_subject import BoxObject, CustomObject, Grasp2Point, PlacePoint, SweepPoint, SweepTask

In [14]:
box1 = pscene.create_object("box1", "box1", BoxObject, hexahedral=True)
box2 = pscene.create_object("box2", "box2", BoxObject, hexahedral=True)
box3 = pscene.create_object("box3", "box3", BoxObject, hexahedral=True)

## Grasp checker

In [15]:
from pkg.planning.filtering.grasp_filter import GraspChecker
gcheck = GraspChecker(pscene, end_link_couple_dict= {"indy0_tcp": ["indy0_tcp", "indy0_link6"], "base_link":["base_link"]})

## Reach SVM

In [16]:
from pkg.planning.filtering.reach_filter import ReachChecker
rcheck = ReachChecker(pscene)

## planners

In [17]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene, [gcheck, rcheck])
mplan.update_gscene()
ppline.set_motion(mplan)

# from pkg.planning.task.a_star import TaskAstar
# tplan = TaskAstar(pscene)
# tplan.prepare()
# ppline.set_sampler(tplan)
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()
ppline.set_sampler(tplan)

## Test plan

In [18]:
from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy

initial_state = pscene.update_state(crob.home_pose)

In [19]:
mplan.reset_log(True)
with GlobalTimer.instance().block("full"):
    goal_nodes = [("goal", "floor", "floor")]
    ppline.search(initial_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, timeout_loop=300, multiprocess=True, timeout=1)
    schedules = ppline.tplan.find_schedules()
    schedules_sorted = ppline.sort_schedule(schedules)
    snode_schedule = ppline.idxSchedule2SnodeScedule(schedules_sorted[0])
    snode_schedule = add_return_motion(snode_schedule)

Use 20/20 agents
node: ('floor', 'floor', 'floor')->('floor', 'floor', 'grip0') = fail
node: ('floor', 'floor', 'floor')->('grip0', 'floor', 'floor') = fail
node: ('floor', 'floor', 'floor')->('grip0', 'floor', 'floor') = fail
node: ('floor', 'floor', 'floor')->('grip0', 'floor', 'floor') = fail
node: ('floor', 'floor', 'floor')->('floor', 'grip0', 'floor') = fail
node: ('floor', 'floor', 'floor')->('floor', 'floor', 'grip0') = fail
node: ('floor', 'floor', 'floor')->('floor', 'floor', 'grip0') = fail
node: ('floor', 'floor', 'floor')->('grip0', 'floor', 'floor') = fail
node: ('floor', 'floor', 'floor')->('floor', 'floor', 'grip0') = fail
node: ('floor', 'floor', 'floor')->('floor', 'floor', 'grip0') = fail
node: ('floor', 'floor', 'floor')->('floor', 'grip0', 'floor') = fail
node: ('floor', 'floor', 'floor')->('floor', 'floor', 'grip0') = fail
node: ('floor', 'floor', 'floor')->('floor', 'floor', 'grip0') = fail
node: ('floor', 'floor', 'floor')->('floor', 'floor', 'grip0') = fail
nod

node: ('floor', 'floor', 'floor')->('floor', 'grip0', 'floor') = fail
node: ('grip0', 'floor', 'floor')->('goal', 'floor', 'floor') = fail
node: ('floor', 'floor', 'floor')->('grip0', 'floor', 'floor') = success
branching: 0->12 (0.31/300.0 s, steps/err: 21(33.0259799957 ms)/0.00170924012085)
node: ('floor', 'floor', 'goal')->('floor', 'grip0', 'goal') = fail
node: ('floor', 'floor', 'grip0')->('floor', 'floor', 'floor') = fail
node: ('floor', 'grip0', 'floor')->('floor', 'goal', 'floor') = fail
node: ('floor', 'grip0', 'floor')->('floor', 'floor', 'floor') = success
node: ('floor', 'grip0', 'floor')->('floor', 'goal', 'floor') = fail
node: ('floor', 'grip0', 'floor')->('floor', 'goal', 'floor') = fail
node: ('floor', 'floor', 'goal')->('floor', 'grip0', 'goal') = fail
branching: 1->13 (0.32/300.0 s, steps/err: 41(46.138048172 ms)/0.00168351404184)
node: ('floor', 'grip0', 'floor')->('floor', 'goal', 'floor') = fail
node: ('floor', 'floor', 'grip0')->('floor', 'floor', 'goal') = fail
n

node: ('floor', 'floor', 'goal')->('grip0', 'floor', 'goal') = fail
node: ('floor', 'floor', 'goal')->('grip0', 'floor', 'goal') = fail
node: ('grip0', 'floor', 'floor')->('goal', 'floor', 'floor') = fail
node: ('floor', 'floor', 'floor')->('floor', 'floor', 'grip0') = fail
node: ('floor', 'floor', 'goal')->('floor', 'grip0', 'goal') = fail
node: ('floor', 'floor', 'grip0')->('floor', 'floor', 'goal') = fail
node: ('floor', 'floor', 'goal')->('floor', 'grip0', 'goal') = fail
node: ('floor', 'floor', 'floor')->('floor', 'floor', 'grip0') = fail
node: ('grip0', 'floor', 'floor')->('goal', 'floor', 'floor') = fail
node: ('floor', 'grip0', 'goal')->('floor', 'goal', 'goal') = fail
node: ('floor', 'grip0', 'floor')->('floor', 'goal', 'floor') = fail
node: ('floor', 'grip0', 'goal')->('floor', 'goal', 'goal') = fail
node: ('floor', 'floor', 'floor')->('grip0', 'floor', 'floor') = fail
node: ('grip0', 'floor', 'floor')->('goal', 'floor', 'floor') = fail
node: ('floor', 'grip0', 'goal')->('flo

node: ('floor', 'floor', 'floor')->('floor', 'floor', 'grip0') = success
======================= terminated 11: first answer acquired from other agent ===============================
=============== try reaching goal =================
branching: 29->39 (0.77/300.0 s, steps/err: 37(52.1461963654 ms)/0.00197535673747)
node: ('goal', 'floor', 'floor')->('grip0', 'floor', 'floor') = success
branching: 34->40 (0.78/300.0 s, steps/err: 41(63.3170604706 ms)/0.00197611893768)
======================= terminated 0: first answer acquired from other agent ===============================
======================= terminated 6: first answer acquired from other agent ===============================
======================= terminated 9: first answer acquired from other agent ===============================
======================= terminated 12: first answer acquired from other agent ===============================
======================= terminated 13: first answer acquired from other agent ============

NameError: name 'add_return_motion' is not defined

node: ('floor', 'floor', 'grip0')->('floor', 'floor', 'goal') = fail
======================= terminated 2: first answer acquired from other agent ===============================
node: ('floor', 'floor', 'goal')->('floor', 'grip0', 'goal') = fail
======================= terminated 7: first answer acquired from other agent ===============================


In [ ]:
for k,v in mplan.result_log.items():
    print("{}: {}% ({})".format(k, np.mean(v)*100, len(v)))

In [ ]:
print(GlobalTimer.instance())

In [48]:
ppline.play_schedule(snode_schedule, period=0.01)

## Text execution

In [21]:
# crob.reset_connection([True])

connection_list
[True]
Connect: Server IP (192.168.0.63)


## unsynced execution

In [23]:
ppline.execute_schedule(snode_schedule, vel_scale=0.5)

binder: floor
rname: None
binder: None
grasp_dict
{'indy0': False}
grasp_seq
[('indy0', False)]
Connect: Server IP (192.168.0.63)
binder: floor
rname: None
binder: None
grasp_dict
{'indy0': False}
grasp_seq
[('indy0', False)]
Connect: Server IP (192.168.0.63)
go
Connect: Server IP (192.168.0.63)
binder: grip0
rname: indy0
binder: None
grasp_dict
{'indy0': True}
grasp_seq
[('indy0', True)]
Connect: Server IP (192.168.0.63)
go
Connect: Server IP (192.168.0.63)
binder: grip0
rname: indy0
binder: brush_face
rname: None
grasp_dict
{'indy0': True}
grasp_seq
[('indy0', True)]
Connect: Server IP (192.168.0.63)
go
Connect: Server IP (192.168.0.63)
binder: grip0
rname: indy0
binder: brush_face
rname: None
grasp_dict
{'indy0': True}
grasp_seq
[('indy0', True)]
Connect: Server IP (192.168.0.63)
go
Connect: Server IP (192.168.0.63)
binder: goal_bd
rname: None
binder: brush_face
rname: None
grasp_dict
{'indy0': False}
grasp_seq
[('indy0', False)]
Connect: Server IP (192.168.0.63)
go
Connect: Server 

## synced execution

In [ ]:
## 
# with RvizPublisher(gscene, []) as rpub:
#     ppline.execute_schedule(snode_schedule, rviz_pub=rpub)

## Disconnect stereo

In [ ]:
stereo.disconnect()